In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
from tqdm import tqdm

from newtonnet.utils.ase_interface import MLAseCalculator
from ase import Atoms, units

In [ ]:
data = dict(np.load('Data/Transition1x/reaction_data/rxn0003.npz'))
irc = [0, -8, -7, -6, -5, -4, -3, -2, -1, 1]
E_irc = data['E'][irc]
R_irc = data['R'][irc]
F_irc = data['F'][irc]
Z_irc = data['Z'][irc]
dR_irc = np.zeros_like(R_irc)
for i in range(1, len(R_irc)):
    dR_irc[i] = R_irc[i] - R_irc[i-1]
dR_abs_irc = np.cumsum(np.linalg.norm(dR_irc, axis=(1,2)))

test_size = 901

R_test = make_interp_spline(np.linspace(0, 1, len(R_irc)), R_irc, k=1)(np.linspace(0, 1, test_size))
Z_test = np.tile(Z_irc[0], (R_test.shape[0], 1))
dR_test = np.zeros_like(R_test)
for i in range(1, test_size):
    dR_test[i] = R_test[i] - R_test[i-1]
dR_abs_test = np.cumsum(np.linalg.norm(dR_test, axis=(1,2)))

In [ ]:
mlcalculator = MLAseCalculator(
    model_path='20230905_output_final/training_44/models/best_model_state.tar',
    settings_path='20230905_output_final/training_44/run_scripts/config050.yml',
    method=None,
    device='cpu',
    )

E_test = np.zeros((R_test.shape[0], 1))
F_test = np.zeros((R_test.shape[0], R_test.shape[1], 3))

for r_test_, r_test in tqdm(enumerate(R_test)):
    molecule = Atoms(numbers=Z_test[0], positions=r_test)
    mlcalculator.calculate(molecule)
    E_test[r_test_] = mlcalculator.results['energy'] / (units.kcal/units.mol)
    F_test[r_test_] = mlcalculator.results['forces'] / (units.kcal/units.mol/units.Ang)
    
print('done!')

In [ ]:
mlcalculator = MLAseCalculator(
    model_path='20230319_TrainedModel_Jerry_ANI_training_1/models/best_model_state.tar',
    settings_path='20230319_TrainedModel_Jerry_ANI_training_1/run_scripts/ani-newtonnet.yml',
    method=None,
    device='cpu',
    )

E_ref = np.zeros((R_test.shape[0], 1))
F_ref = np.zeros((R_test.shape[0], R_test.shape[1], 3))
for r_test_, r_test in tqdm(enumerate(R_test)):
    molecule = Atoms(numbers=Z_test[0], positions=r_test)
    mlcalculator.calculate(molecule)
    E_ref[r_test_] = mlcalculator.results['energy'] / (units.kcal/units.mol)
    F_ref[r_test_] = mlcalculator.results['forces'] / (units.kcal/units.mol/units.Ang)
    
print('done!')

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(dR_abs_irc, E_irc-E_irc[0], 'ok', label='Truth')
plt.plot(dR_abs_test, E_ref-E_irc[0], color='tab:gray', linestyle='--', label='Pre-trained\n (ANI-1)')
plt.plot(dR_abs_test, E_test-E_irc[0], color='tab:red', linestyle='-', label='Fine-tuned\n (T1x)')
plt.xlabel('Reaction coordinate (Å)')
plt.ylabel('Relative energy (kcal/mol)')
plt.legend()
plt.savefig('EnergyFinetuneExample.pdf', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(dR_abs_irc, np.linalg.norm(F_irc, axis=(1,2)), 'ok', label='Truth')
plt.plot(dR_abs_test, np.linalg.norm(F_ref, axis=(1,2)), color='tab:gray', linestyle='--', label='Pre-trained\n (ANI-1)')
plt.plot(dR_abs_test, np.linalg.norm(F_test, axis=(1,2)), color='tab:red', linestyle='-', label='Fine-tuned\n (T1x)')
plt.xlabel('Reaction coordinate (Å)')
plt.ylabel('Force magnitude (kcal/mol/Å)')
plt.legend()
plt.savefig('ForceFinetuneExample.pdf', bbox_inches='tight')
plt.show()